Poisson Equation With Neumann Boundary Conditions
===

Import Netgen/NGSolve Python modules:

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ngsolve import *
from netgen.geom2d import unit_square
import netgen.gui
%gui tk

optfile ./ng.opt does not exist - using default values
togl-version : 2
OCC module loaded
loading ngsolve library
NGSolve-6.2.2305
Using Lapack
Including sparse direct solver UMFPACK
Running parallel using 8 thread(s)


Thank you for using NGSolve


# Problem

$$ \Omega = (0,1)^2 $$

In [ ]:
ud = CoefficientFunction(0)
f = CoefficientFunction(10*exp(-50*((x-0.5)*(x-0.5)+(y-0.5)*(y-0.5))))
g = CoefficientFunction(sin(5*x))
alpha = 5

order = 5
size = 0.01
mesh = Mesh(unit_square.GenerateMesh(maxh=size))


In [ ]:
h

In [ ]:
V = HDiv(mesh, order=order, dirichlet="top|bottom")
Q = L2(mesh, order=order-1)
X = FESpace( [V,Q] )
(sigma, u), (tau, v) = X.TnT()

A = BilinearForm(X, symmetric=True)
A += SymbolicBFI( sigma*tau + div(tau)*u + div(sigma)*v )
A.Assemble()

normal = specialcf.normal(mesh.dim)
F = LinearForm(X)
F += SymbolicLFI(-f*v)
F += SymbolicLFI(g * v, BND)

# f += SymbolicLFI(ud * (tau.Trace()*normal), BND)
F.Assemble()

gfsol = GridFunction(X)
gfsol.vec.data = A.mat.Inverse(X.FreeDofs())*F.vec

Draw(gfsol.components[0], mesh, "sigma")
# Draw(gfsol.components[1], mesh, "u")

approx_exact = gfsol




In [ ]:
print(approx_exact.vec)

In [ ]:
print(approx_exact.vec)

In [ ]:
mesh_size = [0.5, 0.25, 0.125, 0.0625, 0.03125, 0.015625]
order = 2
dofs = []
errors = []
grad_errors = []


for h in mesh_size:
    # Generate mesh for different h for unit_square
    mesh = Mesh(unit_square.GenerateMesh(maxh=h))

    V = HDiv(mesh, order=order, dirichlet="top|bottom")
    Q = L2(mesh, order=order-1)
    X = FESpace( [V,Q] )
    (sigma, u), (tau, v) = X.TnT()

    A = BilinearForm(X, symmetric=True)
    A += SymbolicBFI( sigma*tau + div(tau)*u + div(sigma)*v )
    A.Assemble()

    F = LinearForm(X)
    F += SymbolicLFI(-f*v)
    F += SymbolicLFI(g * v, BND)
    F.Assemble()

    gfsol = GridFunction(X)
    gfsol.vec.data = A.mat.Inverse(X.FreeDofs())*F.vec

    # Compute error
    err = sqrt(Integrate((gfsol.components[1]-approx_exact.components[1])**2, mesh))
    # err = sqrt(Integrate((gfsol.components[1]-approx_exact.components[1])**2, mesh))
    grad_err = sqrt(Integrate((gfsol.components[0]-approx_exact.components[0])**2, mesh))
    print("diff u    = ", sqrt(Integrate((gfsol.components[1]-approx_exact.components[1])**2, mesh)) )
    print("diff flux = ", sqrt(Integrate((gfsol.components[0]-approx_exact.components[0])**2, mesh)) )
    print ("h = ", h, "L2-error:", err, "grad-error:", grad_err)

    errors.append(err)
    grad_errors.append(grad_err)

    

In [ ]:
for i in range(len(mesh_size)-1):
    conv_rate = np.log(errors[i+1]/errors[i])/np.log(mesh_size[i+1]/mesh_size[i])
    print("Convergence rate:", conv_rate)
    print("Convergence rate for grad errors:", np.log(grad_errors[i+1]/grad_errors[i])/np.log(mesh_size[i+1]/mesh_size[i]))

In [ ]:
print(approx_exact.components[0].vec)

In [ ]:
u_xx = CoefficientFunction(2*(y*(1-y))**2*(6*x**2-6*x+1))
u_yy = CoefficientFunction(2*(x*(1-x))**2*(6*y**2-6*y+1))
exact = CoefficientFunction( (x*(1-x)*y*(1-y))**2 )
f2 = -u_xx - u_yy + exact
# grad_exact = CoefficientFunction( (2*x*(1-x)*y*(1-y)*(2*y-1), 2*x*(1-x)*y*(1-y)*(2*x-1)) )
# grad_exact = exact.Diff(x) , exact.Diff(y)


In [ ]:
2**3

In [ ]:
mesh = Mesh(unit_square.GenerateMesh(maxh=0.1))
H = H1(mesh, order=4)
u,v = H.TnT()

A = BilinearForm(H, symmetric=True)
A += SymbolicBFI( grad(u)*grad(v) + u*v )
A.Assemble()

f = LinearForm(H)
f += SymbolicLFI( f2*v )
f.Assemble()

gfu = GridFunction(H)
# gfu.vec.data = A.mat.Inverse(H.FreeDofs())*f.vec
# grad_gfu = grad(gfu)

r = f.vec.CreateVector()
r.data = f.vec - A.mat * gfu.vec
gfu.vec.data += A.mat.Inverse(freedofs=H.FreeDofs()) * r


# Compute error
err = sqrt (Integrate ( (gfu-exact)*(gfu-exact), mesh))
# grad_err = sqrt (Integrate ( (grad_gfu-grad_exact)*(grad_gfu-grad_exact), mesh))
print ("h = ", size, "L2-error:", err)




In [ ]:
print(gfsol.vec.data)

In [ ]:
print("diff u    = ", sqrt(Integrate((gfsol.components[1]-gfu)**2, mesh)) )
print("diff flux = ", sqrt(Integrate((gfsol.components[0]-grad(gfu))**2, mesh)) )
